In [1]:
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

In [2]:
# Hyperparameters
learning_rate = 0.0002
gamma = 0.98

In [25]:
class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        self.data = []

        # policy network 구조: state variable 4개가 들어가 각 action(총 2개)에 대한 probability 도출
        self.fc1 = nn.Linear(4,128)
        self.fc2 = nn.Linear(128,2)
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x), dim=0)

        return x
    
    # policy network parameter update를 위한 episode 데이터에 item : (r, prob[a])를 추가
    def put_data(self,item): 
        self.data.append(item)

    # 쌓아놓은 episode 데이터를 이용하여 policy network 학습
    def train_net(self):
        R=0
        self.optimizer.zero_grad()

        """
        일반적인 pesudocode에서는 t=1 부터 t=T-1으로 순서대로 진행하며 parameter update를 진행.
        그러나 아래 코드에서는 t=T-1부터 T-1 역순으로 진행하며 cumulative return 인 Gt를 간단하게 계산하며 parameter update를 진행
        """
        for r, prob in self.data[::-1]: # data: 실제 sampling한 데이터(episode)
            R = r + gamma * R # episode를 역순으로 진행하기 때문에 이 방식으로 cumulative return을 차례대로 계산할 수 있음 
            """
            DQN에서 했던 것 처럼, optimizer를 통해 gradient ascent(discent)를 사용하기 위해서는 
            gradient 식을 넣는 것이 아니라, loss function을 입력해주어야함.
            REINFORCE에서 loss function에 해당하는 함수는 아래 함수. (미분하면 사용하고자 하는 gradient와 같아짐)
            gradient ascent이므로 -를 붙여줌.
            """
            loss = - R * torch.log(prob)

            """
            한 episode 안에서는 network update를 하지 않으므로, zero_grad()를 하지 않음.
            따라서 아래 loss.backward()는 gradient를 cumulative하게 계산하게 됨
            """
            loss.backward()

        self.optimizer.step() # 위에서 계산한 cumulative gradient를 통해 network update
        self.data = [] # 학습이 끝나면 episode data 초기화

In [26]:
def main():
    env = gym.make('CartPole-v1')
    pi = Policy()
    score = 0.0
    print_interval = 20

    # REINFORCE algorithm
    for n_epi in range(10000):
        # episode 생성 및 데이터 축적
        s = env.reset() # s: initial state
        done = False

        while not done:
            prob = pi(torch.from_numpy(s).float()) # prob: initial state s에서의 policy (action 별 prob.) = Policy(s)
            m = Categorical(prob) # Categorical: prob를 pdf로 변환
            a = m.sample() # 위에서 변환한 pdf에 따라 action 선택 -> torch.Tensor 형태로 출력 -> gradient가 발생되지 않도록 .numpy()나 .item()하여 사용
            s_prime, r, done, info = env.step(a.item()) # 다음 step 진행
            pi.put_data((r,prob[a])) # policy parameter update를 위한 episode에 (r, prob[a])를 추가. prob[a] = Policy(s)[a] = s에서 a를 취할 확률 : s에 대한 정보도 포함??
            s = s_prime
            score += r
        
        # episode 1개당 Policy().train_net() 1회 실행
        pi.train_net()

        if n_epi%print_interval==0 and n_epi!=0:
            print("# of episode :{}, avg score : {}".format(n_epi, score/print_interval))
            score = 0.0
    
    env.close()

In [56]:
main()

# of episode :20, avg score : 19.65
# of episode :40, avg score : 17.1
# of episode :60, avg score : 21.05
# of episode :80, avg score : 25.2
# of episode :100, avg score : 22.95
# of episode :120, avg score : 22.05
# of episode :140, avg score : 23.6
# of episode :160, avg score : 23.95
# of episode :180, avg score : 29.95
# of episode :200, avg score : 29.9
# of episode :220, avg score : 30.55
# of episode :240, avg score : 27.15
# of episode :260, avg score : 30.05
# of episode :280, avg score : 35.2
# of episode :300, avg score : 39.65
# of episode :320, avg score : 41.25
# of episode :340, avg score : 42.05
# of episode :360, avg score : 36.4
# of episode :380, avg score : 44.65
# of episode :400, avg score : 46.7
# of episode :420, avg score : 47.0
# of episode :440, avg score : 53.65
# of episode :460, avg score : 44.6
# of episode :480, avg score : 45.1
# of episode :500, avg score : 51.3
# of episode :520, avg score : 52.0
# of episode :540, avg score : 44.4
# of episode :560,